# 전기톱원숭이 팀

# Private[0.76034] : ESM embedding(3 sequences) + Tabnet

## Import libraries

In [1]:
import os
import gc
import random
import pickle
from itertools import product

import numpy as np
import pandas as pd

import esm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

from tqdm.auto import tqdm

from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MaxAbsScaler, Normalizer, RobustScaler, StandardScaler,MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA


from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.metrics import Metric

## Setting Configuration and Hyperparameters
- Embedding sequence modeling의 정확한 재현을 위해서는 Multi-GPU를 사용하고, Batch_size를 동일하게 적용해야함
- 실제로 운용 시 batch_size를 크게 늘릴 수 있음
- **Don't touch bathc size**

In [2]:
# set visible device
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "0, 1, 2, 3"


CONFIG = {
    'n_worker':16,
    # ESM embedding
    'epitope_max_len':128,
    'antigen_max_len':64,
    'epitope_batch_size':50,
    'antigen_batch_size':100,
    # Feature extraction
    'CT_CTD_features' : False,
    'CNT_features' : False,
    'CT_CTD_PCA':0,
    'CNT_PCA':0,
    # Tabnet model
    'epochs' : 100,
    'patience' : 20,
    'learning_rate':2e-2,
    'weight_decay':1e-5,
    'threshold':0.5,
    'seed':42,
    'fold':5
}

# seed setting function
def seed_everything(seed:int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CONFIG['seed']) # Seed setting

# tabnet params
tabnet_params = dict(
    n_d = 64,   # 8 to 64
    n_a = 128,  # n_d = n_a usally good
    n_steps = 1,
    gamma = 1.3,
    lambda_sparse = 0,
    n_independent = 2,
    n_shared = 1,
    optimizer_fn = optim.Adam,
    optimizer_params = dict(lr = CONFIG['learning_rate'], weight_decay = CONFIG['weight_decay']),
    mask_type = "entmax",
    scheduler_params = dict(
        mode = "min", patience = 5, min_lr = 1e-5, factor = 0.9),
    scheduler_fn = ReduceLROnPlateau,
    seed = CONFIG["seed"],
    verbose = 5
)

# Data Preprocessing

## ESM Embedding feature extraction

- Using Left antigen(64), Epitope(128), Right antigen(64) sequences
- 각 sequence 별 embedding 추출
- 추출된 Embedding feature 파일이 있으면 생략됨. 재현 검증을 원할 시 ```Embeddings/``` 디렉토리 제거 후 실행
- ESM : [ESM Github](https://github.com/facebookresearch/esm)

In [3]:
def protein_sequence_preprocessing(new_df):
    epitope_list = []
    left_antigen_list = []
    right_antigen_list = []
    
    for id_, epitope, antigen, s_p, e_p in tqdm(zip(new_df['id'],new_df['epitope_seq'], new_df['antigen_seq'], new_df['start_position'], new_df['end_position'])):
        start_position = s_p-CONFIG['antigen_max_len']-1
        end_position = e_p+CONFIG['antigen_max_len']
        if start_position < 0:
            start_position = 0
        if end_position > len(antigen):
            end_position = len(antigen)
        
        # left / right antigen sequence 추출
        left_antigen = antigen[int(start_position) : int(s_p)-1]
        right_antigen = antigen[int(e_p) : int(end_position)]

        if CONFIG['epitope_max_len']<len(epitope):
            epitope = epitope[:CONFIG['epitope_max_len']]

        new_epitope = (id_, epitope)
        new_left_antigen = (id_, left_antigen)
        new_right_antigen = (id_, right_antigen)
        
        epitope_list.append(new_epitope)
        left_antigen_list.append(new_left_antigen)
        right_antigen_list.append(new_right_antigen)

    return epitope_list, left_antigen_list, right_antigen_list

### Dataset for ESM input
- ESM 인코더에 입력하기 위한 Dataset 클래스

In [4]:
class ProteinDataset(Dataset):
    def __init__(self, string_labels, protein_list):
        self.string_labels = string_labels
        self.protein_list = protein_list
        
    def __getitem__(self, index):
        self.string_label = self.string_labels[index]
        self.protein = self.protein_list[index]

        return self.string_label, torch.tensor(self.protein, dtype=torch.long)
        
    def __len__(self):
        return len(self.protein_list)

### Train/Test sequence embedding extraction
- Embedding Feature 추출하기 -> 빠른 실행을 위한 train, test set에 적용

In [5]:
# TRAIN_DATA_PATH = "../dataset/train.csv"
# TEST_DATA_PATH = "../dataset/test.csv"
TRAIN_DATA_PATH = "/data/train.csv"
TEST_DATA_PATH = "/data/test.csv"

SAVE_EMBEDDING_DIR = "./Embeddings/"
if not os.path.exists(SAVE_EMBEDDING_DIR):
    os.makedirs(SAVE_EMBEDDING_DIR)

df_train = pd.read_csv(TRAIN_DATA_PATH)
df_test = pd.read_csv(TEST_DATA_PATH)

model, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()
batch_converter = alphabet.get_batch_converter()

train_epitope_list, train_left_list, train_right_list = protein_sequence_preprocessing(df_train)
test_epitope_list, test_left_list, test_right_list = protein_sequence_preprocessing(df_test)

print("finish data loading")

params = {
    "train_epitope":train_epitope_list,
    "train_left":train_left_list,
    "train_right":train_right_list,
    "test_epitope":test_epitope_list,
    "test_left":test_left_list,
    "test_right":test_right_list
}

# load model
model = model.eval()
model = model.cuda()
model = nn.DataParallel(model)      # we use 4 RTX3090 GPUs

for task, protein_list in params.items():
    print(f"== {task} embedding ==")
    file_name = f"{task}_embeddings.pkl"
    file_path = os.path.join(SAVE_EMBEDDING_DIR, file_name)
    
    if os.path.exists(file_path):
        continue
    
    string_labels, _, batch_tokens = batch_converter(protein_list)
    batch_tokens = batch_tokens.numpy()

    dataset = ProteinDataset(string_labels, batch_tokens)
    if task in ["train_epitope", "test_epitope"]:
        dataloader = DataLoader(dataset, batch_size=CONFIG["epitope_batch_size"], shuffle=False, num_workers=CONFIG['n_worker'])
    else:
        dataloader = DataLoader(dataset, batch_size=CONFIG["antigen_batch_size"], shuffle=False, num_workers=CONFIG['n_worker'])

    protein_embeddings = []
    for idx, data in enumerate(tqdm(dataloader)):
        s_labels, b_tokens = data
        b_tokens = b_tokens.cuda()
        with torch.no_grad():
            embeddings = model(b_tokens, repr_layers=[33])
        result = embeddings["representations"][33]
        result = result[:, 0, :]        # use BOS tokens
        result = result.detach().cpu().numpy()
        s_labels = s_labels.numpy()

        for id_, emb_ in zip(s_labels, result):
            protein_embeddings.append((id_, emb_))

    del b_tokens, embeddings

    # save files    
    with open(file_path, "wb") as fw:
        pickle.dump(protein_embeddings, fw)

    print(f"finish {task} embedding extraction")
    
del model

gc.collect()
torch.cuda.empty_cache()

0it [00:00, ?it/s]

0it [00:00, ?it/s]

finish data loading
== train_epitope embedding ==
== train_left embedding ==
== train_right embedding ==
== test_epitope embedding ==
== test_left embedding ==
== test_right embedding ==


## CT-CTD Feature extraction
- 해당 fature 사용을 원할 시 config 파일의 ```CONFIG['CT_CTD_features']=True```로 변경
- 이미 추출된 CT-CTD feature 파일이 있으면 생략됨
- CT : Conjoint Triad features
- CTD : Composition-Transition-Distribution features
- Using 13 physicochemical attributes
- paper : [CT-CTD feature paper](https://www.sciencedirect.com/science/article/pii/S0010482520302973)

In [6]:
## CT-CTD feature extraction
ct_group_dict = {
    'A':1, 'G':1, 'V':1, 'C':2, 'D':3, 'E':3, 'F':4, 'I':4, 'L':4, 'P':4,
    'H':5, 'N':5, 'Q':5, 'W':5, 'K':6, 'R':6, 'M':7, 'S':7, 'T':7, 'Y':7, 
}
trid_groups = list(product(['1', '2', '3', '4', '5', '6', '7'], repeat=3))
trid_dict = []
for trid in trid_groups:
    trid_dict.append(''.join(list(trid)))

ctd_group_dict = [
    ['RKEDQN', 'GASTPHY', 'CLVIMFW'],
    ['QSTNGDE', 'RAHCKMV', 'LYPFIW'],
    ['QNGSWTDERA', 'HMCKV', 'LPFYI'],
    ['KPDESNQT', 'GRHA', 'YMFWLCVI'],
    ['KDEQPSRNTG', 'AHYMLV', 'FIWC'],
    ['RDKENQHYP', 'SGTAW', 'CVLIMF'],
    ['KERSQD', 'NTPG', 'AYHWVMFLIC'],
    ['GASCTPD', 'NVEQIL', 'MHKFRYW'],  
    ['LIFWCMVY', 'PATGS', 'HQRKNED'],
    ['GASDT', 'CPNVEQIL', 'KMHFRYW'],
    ['KR', 'ANCQGHILMFPSTWYV', 'DE'],
    ['EALMQKRH', 'VIYCWFT', 'GNPSD'],
    ['ALFCGIVW', 'RKQEND', 'MPSTHY']
]

new_ctd_group_dict=[]

for attribute in ctd_group_dict:
    new_dict = dict()
    for idx, att in enumerate(attribute):
        for k in att:
            new_dict[k]=idx
    new_ctd_group_dict.append(new_dict)

trans_dict={'01':0, '10':0, '12':1, '21':1, '02':2, '20':2}

def get_CT_CTD_features(acid):
    trid_features = []
    # convert to group
    grouped = ''.join(list(map(lambda x:str(ct_group_dict[x]), acid)))

    # ====== extract CT
    for trid in trid_dict:
        trid_features.append(grouped.count(trid))

    # ====== extract CTD
    comp_features = []
    trans_features = []
    dist_features = []

    for attribute in new_ctd_group_dict:
        # ==== Compositon feature
        comps_f = []
        comps = ''.join(list(map(lambda x:str(attribute[x]), acid)))
        try:
            comps_f.append(comps.count('0')/len(acid))
            comps_f.append(comps.count('1')/len(acid))
            comps_f.append(comps.count('2')/len(acid))
        except:
            # division 0
            comps_f=[0, 0, 0]
        
        # ==== Transition feature
        trans_f = [0, 0, 0]
        for idx in range(len(comps)-1):
            try:
                trans = comps[idx:idx+2]
                trans_f[trans_dict[trans]]+=1
            except:
                continue

        if len(comps)==0:
            trans_f = [0, 0, 0]
        else:
            try:
                trans_f[0]/=len(comps)-1
                trans_f[1]/=len(comps)-1
                trans_f[2]/=len(comps)-1
            except:
                trans_f = [0, 0, 0]

        # ==== Distribution feature
        dist_f = []
        np_comps = np.array(list(comps))

        for pos in ['0', '1', '2']:
            g_pos = np.where(np_comps==pos)[0]
            try:
                g_pos -=g_pos[0]
            except:
                dist_f.extend([0, 0, 0, 0, 0])
                continue
            g_pos += 1
            g_pos = g_pos.tolist()
            pos_len = len(g_pos)
            
            dist_f.append(1/len(comps))
            dist_f.append(g_pos[int(0.25*pos_len)-1]/len(comps))
            dist_f.append(g_pos[int(0.5*pos_len)-1]/len(comps))
            dist_f.append(g_pos[int(0.75*pos_len)-1]/len(comps))
            dist_f.append(g_pos[int(pos_len)-1]/len(comps))
        

        comp_features.append(comps_f)
        trans_features.append(trans_f)
        dist_features.append(dist_f)


    return trid_features, comp_features, trans_features, dist_features

def extract_ct_ctd_features(df):
    result_features = []
    
    for epitope, antigen in tqdm(zip(df['epitope_seq'], df['antigen_seq'])):
        length_features = [len(epitope), len(antigen)]

        # CT, CTD features
        epit_ct, epit_comp, epit_trans, epit_dist = get_CT_CTD_features(epitope)
        anti_ct, anti_comp, anti_trans, anti_dist = get_CT_CTD_features(antigen)

        ct_ctd_features = []

        ct_ctd_features+= epit_ct
        for arr in [epit_comp, epit_trans, epit_dist]:
            ct_ctd_features += np.array(arr).reshape(-1).tolist()

        ct_ctd_features+= anti_ct
        for arr in [anti_comp, anti_trans, anti_dist]:
            ct_ctd_features += np.array(arr).reshape(-1).tolist()

        extracted_features = length_features + ct_ctd_features
        result_features.append(extracted_features)
    
    result_features = np.array(result_features)

    return result_features


TRAIN_CT_CTD_PATH = f"train_epitope_antigen_CTCTD.pkl"
TEST_CT_CTD_PATH = f"test_epitope_antigen_CTCTD.pkl"
CT_CTD_DIR = "CT_CTD_Features"

if CONFIG['CT_CTD_features']:
    if not os.path.exists(CT_CTD_DIR):
        os.makedirs(CT_CTD_DIR)

    train_ct_ctd_path = os.path.join(CT_CTD_DIR, TRAIN_CT_CTD_PATH)
    test_ct_ctd_path = os.path.join(CT_CTD_DIR, TEST_CT_CTD_PATH)

    if os.path.exists(train_ct_ctd_path) and os.path.exists(test_ct_ctd_path):
        pass
    else:
        train_ct_ctd_features = extract_ct_ctd_features(df_train)
        test_ct_ctd_features = extract_ct_ctd_features(df_test)

        # Save
        with open(train_ct_ctd_path, "wb") as fw:
            pickle.dump(train_ct_ctd_features, fw)

        with open(test_ct_ctd_path, "wb") as fw:
            pickle.dump(test_ct_ctd_features, fw)
            
        del train_ct_ctd_features
        del test_ct_ctd_features

        print(f"finished extract CT-CTD features")

## Counting features : Amino acids / Dipeptides
- 해당 fature 사용을 원할 시 config 파일의 ```CONFIG['CNT_features']=True```로 변경
- 이미 추출된 CNT feature 파일이 있으면 생략됨
- Frequency feature of amino acids and dipeptidse in protein sequences
- Reference : [FEGS](https://bmcbioinformatics.biomedcentral.com/track/pdf/10.1186/s12859-021-04223-3.pdf)

In [7]:
## CNT feature extraction
protein_list = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
combs_protein_list = []

seques =  list(product(protein_list, repeat=2))
for comb in seques:
    combs_protein_list.append(''.join(list(comb)))


def get_counting_features(counting_dicts, protein_seq):
    single_dict, double_dict = counting_dicts
    left_antigen, epitope, right_antigen = protein_seq

    left_single, epitope_single, right_single = [], [], []
    left_double, epitope_double, right_double = [], [], []

    for single_acid in single_dict:
        left_single.append(left_antigen.count(single_acid)) 
        epitope_single.append(epitope.count(single_acid))
        right_single.append(right_antigen.count(single_acid)) 

    for double_acid in double_dict:
        left_double.append(left_antigen.count(double_acid)) 
        epitope_double.append(epitope.count(double_acid))
        right_double.append(right_antigen.count(double_acid)) 

    counting_features = left_single + epitope_single + right_single + left_double + epitope_double + right_double

    return counting_features

def extract_cnt_features(df):
    result_features = []
    
    for epitope, antigen, s_p, e_p in tqdm(zip(df['epitope_seq'], df['antigen_seq'], df['start_position'], df['end_position'])):
        left_antigen = antigen[: int(s_p)-1]
        right_antigen = antigen[int(e_p) : ]

        # counting features
        counting_features = get_counting_features((protein_list, combs_protein_list), (left_antigen, epitope, right_antigen))
        result_features.append(counting_features)
    
    result_features = np.array(result_features)

    return result_features


TRAIN_CNT_PATH = f"train_epitope_antigen_CNT.pkl"
TEST_CNT_PATH = f"test_epitope_antigen_CNT.pkl"
CNT_DIR = "CNT_Features"


if CONFIG['CNT_features']:

    if not os.path.exists(CNT_DIR):
        os.makedirs(CNT_DIR)

    train_cnt_path = os.path.join(CNT_DIR, TRAIN_CNT_PATH)
    test_cnt_path = os.path.join(CNT_DIR, TEST_CNT_PATH)

    if os.path.exists(train_cnt_path) and os.path.exists(test_cnt_path):
        pass
    else:
        train_cnt_features = extract_cnt_features(df_train)
        test_cnt_features = extract_cnt_features(df_test)
        # Save
        with open(train_cnt_path, "wb") as fw:
            pickle.dump(train_cnt_features, fw)

        with open(test_cnt_path, "wb") as fw:
            pickle.dump(test_cnt_features, fw)

        del train_cnt_features
        del test_cnt_features

        print(f"finished extract CNT features")

### Norm and PCA for dimension reduction
- 추출된 feature의 normalization과 dimension reduction을 위한 함수
- scaler와 pca가 pickle형태로 저장됨. test 시 해당 파일 불러와서 적용 가능
- Tabnet의 평가지표로 F1-macro로 사용하기 위한 커스텀 메트릭 클래스

In [8]:
# ============= Norm 
def norm_transform(datatype, data, scaler_name='z-score', scaler=None):
    scaler_dict = {
        'z-score':StandardScaler(),
        'minmax':MinMaxScaler(),
        'maxabs':MaxAbsScaler(),
        'robust':RobustScaler(),
        'norm':Normalizer()
    }
    
    # use only train
    if not datatype=="test":
        scaler = scaler_dict[scaler_name]
        scaled_train = scaler.fit_transform(data)
        return scaled_train, scaler
    else:
        scaled_test = scaler.transform(data)
        return scaled_test

# ============= pca 
def pca_transform(datatype, data, n_comp=300, pca=None):
    if not datatype=="test":
        pca = PCA(n_components=n_comp, random_state=CONFIG["seed"])
        pca_train = pca.fit_transform(data)
        print(f"with {n_comp} components, pca variance ratio : {sum(pca.explained_variance_ratio_)}")
        return pca_train, pca
    else:
        pca_test = pca.transform(data)
        return pca_test

class F1_macro(Metric):
    def __init__(self):
        self._name = "f1_macro"
        self._maximize = True

    def __call__(self, y_true, y_score):
        y_score = np.argmax(y_score, axis=1)
        f1 = f1_score(y_true, y_score, average="macro")
        return f1

# Model Traning (Tabnet)
-  Tabnet model : [Tabnet Github](https://github.com/dreamquark-ai/tabnet)
-  Ebmedding feature preprocessing 함수
-  left antigen, epitope, right antigen embedding feature와 category feature를 합쳐서 출력함

In [9]:
# for tabnet data
def get_preprocessing(data_type, epitope_data, left_anti, right_anti, new_df):
    tab_net_feature_list = []

    features_cols = ["assay_method_technique", "assay_group", "disease_type", "disease_state", "reference_date", "reference_journal", "reference_title"]
    feature_data = enc.transform(new_df[features_cols]).toarray()

    # sort by id
    epitope_data.sort()
    left_anti.sort()
    right_anti.sort()
    
    for df_id, epi_embs, left_embs, right_embs, feature in tqdm(zip(new_df['id'], epitope_data, left_anti, right_anti, feature_data)):
        if not df_id == epi_embs[0]:
            print("Not matched ID")
        tab_net_features = np.append(epi_embs[1], left_embs[1])
        tab_net_features = np.append(tab_net_features , right_embs[1])
        tab_net_features = np.append(tab_net_features , feature)
        tab_net_feature_list.append(tab_net_features)
    
    label_list = None
    if data_type != 'test':
        label_list = []
        for label in new_df['label']:
            label_list.append(label)
        label_list = np.array(label_list)
    print(f'{data_type} dataframe preprocessing was done.')

    tab_net_feature_list = np.array(tab_net_feature_list)
    
    return tab_net_feature_list, label_list

### Training

- Using ESM embedding of **left antigen, epitope, right antigen**
- Categrical features to one-hot encoding
- if you want to use other features, set ```CONFIG['CT_CTD_features']=True``` or ```CONFIG['CNT_features']=True```
- if you want to use PCA for features, set ```CONFIG['CT_CTC_PCA]``` or ```CONFIG['CNT_PCA]```

#### 5-fold for validation and use mean of prediction logits

In [10]:
# PATH
EPITOPE_EMB_PATH = "./Embeddings/train_epitope_embeddings.pkl"
LEFT_EMB_PATH = "./Embeddings/train_left_embeddings.pkl"
RIGHT_EMB_PATH = "./Embeddings/train_right_embeddings.pkl"
TEST_EPITOPE_EMB_PATH = "./Embeddings/test_epitope_embeddings.pkl"
TEST_LEFT_EMB_PATH = "./Embeddings/test_left_embeddings.pkl"
TEST_RIGHT_EMB_PATH = "./Embeddings/test_right_embeddings.pkl"


MODEL_DIR_NAME = "./Tabnet_ESM_models"

if not os.path.exists(MODEL_DIR_NAME):
    os.makedirs(MODEL_DIR_NAME)
    
df_train = pd.read_csv(TRAIN_DATA_PATH)
df_test = pd.read_csv(TEST_DATA_PATH)


# Load extraction features

# load ESM embeddings  [ 1280 x 3 ] : [ 3840 features]
with open(EPITOPE_EMB_PATH, "rb") as fr:
    epitope_data = pickle.load(fr)
with open(LEFT_EMB_PATH, "rb") as fr:
    left_anti_data = pickle.load(fr)
with open(RIGHT_EMB_PATH, "rb") as fr:
    right_anti_data = pickle.load(fr)

# Test
with open(TEST_EPITOPE_EMB_PATH, "rb") as fr:
    test_epitope_data = pickle.load(fr)
with open(TEST_LEFT_EMB_PATH, "rb") as fr:
    test_left_anti_data = pickle.load(fr)
with open(TEST_RIGHT_EMB_PATH, "rb") as fr:
    test_right_anti_data = pickle.load(fr)



# label one hot encoding
enc = OneHotEncoder(handle_unknown='ignore')
features_cols = ["assay_method_technique", "assay_group", "disease_type", "disease_state", "reference_date", "reference_journal", "reference_title"]
enc.fit(df_train[features_cols])

tab_net_feature_list, label_list = get_preprocessing('train', epitope_data, left_anti_data, right_anti_data, df_train)
test_tab_net_feature_list, test_label_list = get_preprocessing('test', test_epitope_data, test_left_anti_data, test_right_anti_data, df_test)


# CT-CTD features [ 3 + (616 x 3) ] : [1851 features]
if CONFIG['CT_CTD_features']:
    TRAIN_CT_CTD_PATH = f"train_epitope_antigen_CTCTD.pkl"
    TEST_CT_CTD_PATH = f"test_epitope_antigen_CTCTD.pkl"
    CT_CTD_SCALER = f"CTCTD_scaler.pkl"
    CT_CTD_PCA = f"CTCTD_pca.pkl"
    CT_CTD_DIR = "CT_CTD_Features"

    train_ct_ctd_path = os.path.join(CT_CTD_DIR, TRAIN_CT_CTD_PATH)
    test_ct_ctd_path = os.path.join(CT_CTD_DIR, TEST_CT_CTD_PATH)
    scaler_ct_ctd_path = os.path.join(CT_CTD_DIR, CT_CTD_SCALER)
    pca_ct_ctd__path = os.path.join(CT_CTD_DIR, CT_CTD_PCA)

    with open(train_ct_ctd_path, "rb") as fr:
        train_ct_ctd_features = pickle.load(fr)

    with open(test_ct_ctd_path, "rb") as fr:
        test_ct_ctd_features= pickle.load(fr)

    # Norm
    train_ct_ctd_features, scaler = norm_transform("train", train_ct_ctd_features, "minmax")
    test_ct_ctd_features = norm_transform("test", test_ct_ctd_features, "minmax", scaler)
    # need to save norm scaler
    print(f"CT-CTD feature normalization finished")

    with open(scaler_ct_ctd_path, "wb") as fw:
        pickle.dump(scaler, fw)

    # PCA
    if not CONFIG['CT_CTD_PCA']==0:
        n_comps = CONFIG['CT_CTD_PCA']
        before_features = train_ct_ctd_features.shape[1]
        train_ct_ctd_features, pca = pca_transform("train", train_ct_ctd_features, n_comps)
        test_ct_ctd_features = pca_transform("test", test_ct_ctd_features, n_comps, pca)
        # need to save pca scaler
        print(f"CT_CTD feature decomposition finished {before_features} -> {n_comps}")

        with open(pca_ct_ctd__path, "wb") as fw:
            pickle.dump(pca, fw)

    tab_net_feature_list = np.concatenate((train_ct_ctd_features, tab_net_feature_list), axis=1)
    test_tab_net_feature_list = np.concatenate((test_ct_ctd_features, test_tab_net_feature_list), axis=1)


# CNT features [ 420 x 3 ] : [1260 features]
if CONFIG['CNT_features']:
    TRAIN_CNT_PATH = f"train_epitope_antigen_CNT.pkl"
    TEST_CNT_PATH = f"test_epitope_antigen_CNT.pkl"
    CNT_SCALER = f"CNT_scaler.pkl"
    CNT_PCA = f"CNT_pca.pkl"
    CNT_DIR = "CNT_Features"
    train_cnt_path = os.path.join(CNT_DIR, TRAIN_CNT_PATH)
    test_cnt_path = os.path.join(CNT_DIR, TEST_CNT_PATH)
    scaler_cnt_path = os.path.join(CNT_DIR, CNT_SCALER)
    pca_cnt_path = os.path.join(CNT_DIR, CNT_PCA)
    
    with open(train_cnt_path, "rb") as fr:
        train_cnt_features= pickle.load(fr)

    with open(test_cnt_path, "rb") as fr:
        test_cnt_features= pickle.load(fr)
    # Norm
    train_cnt_features, scaler = norm_transform("train", train_cnt_features, "minmax")
    test_cnt_features = norm_transform("test", test_cnt_features, "minmax", scaler)
    # need to save norm scaler

    print(f"CNT feature normalization finished")
    with open(scaler_cnt_path, "wb") as fw:
        pickle.dump(scaler, fw)


    # PCA
    if not CONFIG['CNT_PCA']==0:
        n_comps = CONFIG['CNT_PCA']
        before_features = train_cnt_features.shape[1]
        train_cnt_features, pca = pca_transform("train", train_cnt_features, n_comps)
        test_cnt_features = pca_transform("test", test_cnt_features, n_comps, pca)
        # need to save pca scaler
        print(f"CNT feature decomposition finished {before_features.shape[1]} -> {n_comps}")
        
        with open(pca_cnt_path, "wb") as fw:
            pickle.dump(pca, fw)

    tab_net_feature_list = np.concatenate((train_cnt_features, tab_net_feature_list), axis=1)
    test_tab_net_feature_list = np.concatenate((test_cnt_features, test_tab_net_feature_list), axis=1)


kf = KFold(n_splits=CONFIG['fold'],  random_state=CONFIG['seed'], shuffle=True)
avg_loss, avg_f1 = 0, 0

LOG_PATH= os.path.join(MODEL_DIR_NAME,"log.txt")

with open(LOG_PATH, "w") as fw:
    fw.write("========================================\n")
    fw.write("=   Tabnet model with ESM embeddings   =\n")
    fw.write("=--------------------------------------=\n")
    fw.write("=        128 epitope embeddings        =\n")
    fw.write("=      64 left antigen embeddings      =\n")
    fw.write("=      64 right antigen embeddings     =\n")
    fw.write("========================================\n\n")

    for fold, (train_idx, test_idx) in enumerate(kf.split(tab_net_feature_list)):
        fw.write(f": ========= FOLD {fold+1} ========= :\n")

        model = TabNetClassifier(**tabnet_params)
        model.fit(X_train=tab_net_feature_list[train_idx], y_train=label_list[train_idx],
                    eval_set=[(tab_net_feature_list[test_idx], label_list[test_idx])],
                    max_epochs=CONFIG['epochs'], patience=CONFIG['patience'],eval_metric=["auc", "accuracy", F1_macro])

        # save model
        model_name = f"./tabnet_esm_model_fold{fold+1}"
        model_path = os.path.join(MODEL_DIR_NAME, model_name)
        model.save_model(model_path)

        # save history
        history = model.history
        loss, auc, acc, f1 = history['loss'], history['val_0_auc'], history['val_0_accuracy'], history['val_0_f1_macro']
        
        best_loss, best_f1 = 0, 0

        for idx, (l, au, ac, f) in enumerate(zip(loss, auc, acc, f1)):
            if (idx+1)%5==0:
                fw.write(f":Epoch {idx+1:2d} :: loss {l:.6f}\t val Auc {au:.6f}\t val Acc {ac:.6f}\t val F1-macro: {f:.6f}\n")
            if best_f1 < f:
                best_f1 = f
                best_loss = l
        fw.write(f"\n::BEST VALID::  loss : {best_loss:.6f}\t F1-macro: {best_f1:6f}:\n\n")

        avg_loss += best_loss
        avg_f1 += best_f1
        del model

    fw.write(f"\n:== AVG 10 FOLD  loss : {avg_loss/5:.6f}\t F1-macro: {avg_f1/5:6f} ==:\n")


0it [00:00, ?it/s]

train dataframe preprocessing was done.


0it [00:00, ?it/s]

test dataframe preprocessing was done.
Device used : cuda
epoch 0  | loss: 0.21128 | val_0_auc: 0.9059  | val_0_accuracy: 0.91282 | val_0_f1_macro: 0.48019 |  0:00:05s
epoch 5  | loss: 0.12062 | val_0_auc: 0.82127 | val_0_accuracy: 0.91667 | val_0_f1_macro: 0.60592 |  0:00:33s
epoch 10 | loss: 0.10866 | val_0_auc: 0.80546 | val_0_accuracy: 0.91612 | val_0_f1_macro: 0.59483 |  0:01:00s
epoch 15 | loss: 0.09735 | val_0_auc: 0.97191 | val_0_accuracy: 0.95632 | val_0_f1_macro: 0.84935 |  0:01:28s
epoch 20 | loss: 0.08983 | val_0_auc: 0.97191 | val_0_accuracy: 0.9554  | val_0_f1_macro: 0.84411 |  0:01:55s
epoch 25 | loss: 0.08432 | val_0_auc: 0.96988 | val_0_accuracy: 0.95556 | val_0_f1_macro: 0.84927 |  0:02:23s
epoch 30 | loss: 0.08105 | val_0_auc: 0.97311 | val_0_accuracy: 0.95629 | val_0_f1_macro: 0.8529  |  0:02:50s
epoch 35 | loss: 0.0765  | val_0_auc: 0.97262 | val_0_accuracy: 0.95697 | val_0_f1_macro: 0.85816 |  0:03:18s
epoch 40 | loss: 0.06908 | val_0_auc: 0.97087 | val_0_accuracy

## Model inference with 5-fold tabnet models

In [11]:
preds_logits = np.zeros((len(df_test), 2))

for fold in range(CONFIG['fold']):
    model_path = os.path.join(MODEL_DIR_NAME, f"tabnet_esm_model_fold{fold+1}.zip")
    tabnet_model = TabNetClassifier(**tabnet_params)
    tabnet_model.load_model(model_path)

    preds_logits+= tabnet_model.predict_proba(test_tab_net_feature_list)

preds_logits/=5.
preds = preds_logits[:, 1]
pred_label = np.where(preds>CONFIG['threshold'], 1, 0)
uni, cnt = np.unique(pred_label, return_counts=True)
print(f"Predict Label Dist : {cnt}")

# submit = pd.read_csv('../dataset/sample_submission.csv')
submit = pd.read_csv('/data/sample_submission.csv')
submit['label'] = pred_label
submit.to_csv('./tabnet_submit.csv', index=False)

Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Predict Label Dist : [107563  13381]
